# Document Similarity using NLTK and gensim

### tokenization, word counts, and possibly calculated tf-idf scores for words

### Tokenization in NLTK

In [1]:
import nltk
import string

from collections import Counter
import urllib

# nltk 코퍼스 파일 받아오고 열기
text = nltk.corpus.gutenberg.raw("burgess-busterbrown.txt")

# raw text를 모두 소문자로 바꾸고 특수문자를 제거한 뒤 tokenize하기
def get_tokens(text):
    lowers = text.lower()
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    return tokens

tokens = get_tokens(text)
print(tokens)


['The', 'Adventures', 'of', 'Buster', 'Bear', 'by', 'Thornton', 'W', 'Burgess', '1920', 'I', 'BUSTER', 'BEAR', 'GOES', 'FISHING', 'Buster', 'Bear', 'yawned', 'as', 'he', 'lay', 'on', 'his', 'comfortable', 'bed', 'of', 'leaves', 'and', 'watched', 'the', 'first', 'early', 'morning', 'sunbeams', 'creeping', 'through', 'the', 'Green', 'Forest', 'to', 'chase', 'out', 'the', 'Black', 'Shadows', 'Once', 'more', 'he', 'yawned', 'and', 'slowly', 'got', 'to', 'his', 'feet', 'and', 'shook', 'himself', 'Then', 'he', 'walked', 'over', 'to', 'a', 'big', 'pine', 'tree', 'stood', 'up', 'on', 'his', 'hind', 'legs', 'reached', 'as', 'high', 'up', 'on', 'the', 'trunk', 'of', 'the', 'tree', 'as', 'he', 'could', 'and', 'scratched', 'the', 'bark', 'with', 'his', 'great', 'claws', 'After', 'that', 'he', 'yawned', 'until', 'it', 'seemed', 'as', 'if', 'his', 'jaws', 'would', 'crack', 'and', 'then', 'sat', 'down', 'to', 'think', 'what', 'he', 'wanted', 'for', 'breakfast', 'While', 'he', 'sat', 'there', 'trying'

In [2]:
# tokenize된 문서 안에서 각 단어의 빈도 세기
count = Counter(tokens)

# 가장 많이 쓰인 단어부터 상위 10개 확인
print(count.most_common(10))

[('the', 639), ('he', 562), ('and', 484), ('to', 426), ('of', 326), ('that', 285), ('a', 275), ('was', 274), ('it', 239), ('Buster', 216)]


In [3]:
# nltk를 이용해 stopword 제거하기 
from nltk.corpus import stopwords

tokens = get_tokens(text)
filtered = [w for w in tokens if not w in stopwords.words('english')]

# stopword가 제거된 토큰들의 빈도 세기
count = Counter(filtered)

# 가장 많이 쓰인 토큰부터 상위 100개 확인
print(count.most_common(100))

[('Buster', 216), ('I', 156), ('Bear', 137), ('He', 116), ('Little', 111), ('Joe', 108), ('one', 99), ('Brown', 95), ('Farmer', 94), ('little', 92), ('boy', 88), ('Green', 81), ('see', 73), ('Forest', 66), ('It', 61), ('could', 56), ('Blacky', 51), ('said', 50), ('know', 48), ('Otter', 47), ('time', 47), ('would', 46), ('way', 44), ('fish', 44), ('Then', 43), ('But', 43), ('berries', 43), ('great', 41), ('Sammy', 41), ('afraid', 40), ('away', 40), ('Billy', 39), ('looked', 39), ('pail', 39), ('big', 38), ('trout', 37), ('You', 36), ('Old', 36), ('tree', 35), ('get', 33), ('Brook', 32), ('right', 32), ('Laughing', 30), ('And', 30), ('Now', 30), ('thing', 30), ('Chatterer', 28), ('Mink', 28), ('make', 27), ('pool', 27), ('Pasture', 27), ('So', 26), ('think', 25), ('thought', 25), ('frightened', 25), ('Jay', 24), ('go', 24), ('sure', 24), ('saw', 24), ('back', 24), ('bad', 24), ('There', 24), ('made', 24), ('heard', 23), ('else', 23), ('went', 23), ('That', 23), ('even', 23), ('head', 23)

In [4]:
# 각 토큰의 stem 얻기

from nltk.stem.porter import *

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)

# stemmize된 토큰들의 빈도 세기
count = Counter(stemmed)

# 가장 많이 쓰인 stem부터 상위 100개 확인
print(count.most_common(100))

[('buster', 229), ('littl', 207), ('I', 156), ('bear', 152), ('He', 116), ('joe', 112), ('one', 103), ('brown', 102), ('farmer', 100), ('boy', 95), ('green', 87), ('see', 78), ('fish', 69), ('look', 68), ('forest', 66), ('It', 61), ('could', 56), ('blacki', 52), ('know', 52), ('berri', 52), ('great', 51), ('pool', 51), ('otter', 51), ('time', 51), ('laugh', 50), ('said', 50), ('big', 49), ('would', 46), ('way', 45), ('thing', 45), ('then', 43), ('but', 43), ('pail', 43), ('make', 42), ('sammi', 42), ('get', 42), ('come', 42), ('old', 42), ('tree', 41), ('go', 40), ('afraid', 40), ('away', 40), ('billi', 39), ('trout', 37), ('right', 36), ('you', 36), ('brook', 32), ('and', 32), ('chatter', 30), ('now', 30), ('think', 29), ('thought', 29), ('head', 28), ('eat', 28), ('mink', 28), ('smile', 28), ('sure', 27), ('frighten', 27), ('run', 27), ('pastur', 27), ('So', 26), ('jay', 25), ('thief', 25), ('even', 25), ('made', 25), ('els', 24), ('saw', 24), ('back', 24), ('bad', 24), ('there', 24)

### Tf-Idf in gensim
###### gensim 설치 명령: pip install --upgrade gensim
###### gensim installation instructions: https://radimrehurek.com/gensim/install.html

In [11]:
import nltk
import string
import gensim

# 지금까지 했던 tokenize, stopword 제거, punctuation 제거, stemmize를 모두 수행하는 새 tokenize 함수
def tokenize(text):
    tokens = get_tokens(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

# nltk gutenberg 텍스트의 목록 불러오기
titles = nltk.corpus.gutenberg.fileids()
print(titles)

# 목록에 있는 텍스트를 순서대로 tokenize(stemize)한 뒤 하나의 리스트로 모으기
text_tokens = []
for title in titles:
    text = nltk.corpus.gutenberg.raw(title)
    tokens = tokenize(text)
    text_tokens.append(tokens)
    

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [6]:
# 각 리스트에 들어 있는 tokenized text의 형태
print(text_tokens[0])

['emma', 'by', 'jane', 'austen', '1816', 'volum', 'I', 'chapter', 'I', 'emma', 'woodhous', 'handsom', 'clever', 'and', 'rich', 'with', 'a', 'comfort', 'home', 'and', 'happi', 'disposit', 'seem', 'to', 'unit', 'some', 'of', 'the', 'best', 'bless', 'of', 'exist', 'and', 'had', 'live', 'nearli', 'twenti', 'one', 'year', 'in', 'the', 'world', 'with', 'veri', 'littl', 'to', 'distress', 'or', 'vex', 'her', 'she', 'wa', 'the', 'youngest', 'of', 'the', 'two', 'daughter', 'of', 'a', 'most', 'affection', 'indulg', 'father', 'and', 'had', 'in', 'consequ', 'of', 'her', 'sister', 's', 'marriag', 'been', 'mistress', 'of', 'hi', 'hous', 'from', 'a', 'veri', 'earli', 'period', 'her', 'mother', 'had', 'die', 'too', 'long', 'ago', 'for', 'her', 'to', 'have', 'more', 'than', 'an', 'indistinct', 'remembr', 'of', 'her', 'caress', 'and', 'her', 'place', 'had', 'been', 'suppli', 'by', 'an', 'excel', 'woman', 'as', 'gover', 'who', 'had', 'fallen', 'littl', 'short', 'of', 'a', 'mother', 'in', 'affect', 'sixtee

In [7]:
# gensim 패키지를 이용하여 단어들을 숫자 인덱스로 바꾸어 쓰기
# 딕셔너리 형태로 저장

from gensim import corpora
dictionary = corpora.Dictionary(text_tokens)

# 인덱스로 단어 호출하기 
print(dictionary[10])
print(dictionary[100])

# 단어로 인덱스 호출하기 
print(dictionary.token2id['road'])
print(dictionary.token2id['like'])

Ah
_evening_
3521
2543


In [14]:
# 딕셔너리 내 단어들의 총 개수 
print("Number of words in dictionary:",len(dictionary))

# 각 인덱스가 부여된 단어들의 목록 
for i in dictionary.keys():
    print(i, dictionary[i])

Number of words in dictionary: 26306
0 000
1 10
2 1816
3 23rd
4 24th
5 26th
6 28th
7 7th
8 8th
9 A
10 Ah
11 Am
12 An
13 As
14 At
15 Be
16 By
17 C
18 Do
19 Dr
20 E
21 Em
22 F
23 Go
24 Ha
25 He
26 I
27 II
28 IV
29 IX
30 If
31 In
32 Is
33 It
34 K
35 La
36 M
37 MY
38 Ma
39 Me
40 Mr
41 My
42 N
43 No
44 Of
45 Oh
46 On
47 Or
48 S
49 So
50 St
51 To
52 V
53 VI
54 W
55 We
56 X
57 XI
58 XV
59 _
60 _______
61 _a_
62 _accepted_
63 _adair_
64 _addition_
65 _all_
66 _almost_
67 _alone_
68 _amor_
69 _and_
70 _answer_
71 _any_
72 _appropriation_
73 _as_
74 _assistance_
75 _at_
76 _bath_
77 _be_
78 _been_
79 _blunder_
80 _boiled_
81 _both_
82 _bride_
83 _broke_
84 _caro_
85 _cause_
86 _chaperon_
87 _compassion_
88 _compliments_
89 _court_
90 _courtship_
91 _did_
92 _dissolved_
93 _dixon_
94 _dixons_
95 _doubts_
96 _each_
97 _eighteen_
98 _elton_
99 _engagement_
100 _evening_
101 _felt_
102 _first_
103 _gentleman_
104 _great_
105 _greater_
106 _had_
107 _half_
108 _happily_
109 _has_
110 _have_
111 _he_


973 concept
974 concern
975 concert
976 concess
977 concili
978 concis
979 conclud
980 conclus
981 concurr
982 condemn
983 condescend
984 condescens
985 condit
986 condol
987 conduc
988 conduct
989 confeder
990 confer
991 confess
992 confid
993 confidant
994 confidenti
995 confin
996 confirm
997 confound
998 confus
999 congratul
1000 congratulatori
1001 conjectur
1002 conjug
1003 connect
1004 connexion
1005 conniv
1006 conquer
1007 conquest
1008 conscienc
1009 consciou
1010 conscious
1011 consent
1012 consequ
1013 conservatori
1014 consid
1015 consider
1016 consist
1017 consol
1018 constanc
1019 constant
1020 constantli
1021 constern
1022 constitut
1023 constrain
1024 construct
1025 consult
1026 consumpt
1027 contain
1028 contempl
1029 contempt
1030 contemptu
1031 contend
1032 content
1033 conting
1034 continu
1035 contract
1036 contradict
1037 contrari
1038 contrast
1039 contribut
1040 contrit
1041 contriv
1042 controul
1043 conundrum
1044 conveni
1045 convers
1046 convey
1047 convict

1973 greatcoat
1974 greater
1975 greatest
1976 greatli
1977 green
1978 greensward
1979 greet
1980 grew
1981 grey
1982 grief
1983 griev
1984 grievanc
1985 grievou
1986 grievous
1987 gross
1988 grossli
1989 ground
1990 groundless
1991 groundwork
1992 group
1993 grove
1994 grow
1995 grown
1996 grudg
1997 gruel
1998 guard
1999 guess
2000 guest
2001 guid
2002 guidabl
2003 guidanc
2004 guileless
2005 guilt
2006 guilti
2007 guinea
2008 ha
2009 haberdash
2010 habit
2011 habitu
2012 had
2013 hair
2014 hairdress
2015 half
2016 hall
2017 hand
2018 handl
2019 handsom
2020 handsomest
2021 handwrit
2022 hang
2023 hannah
2024 haphazard
2025 happen
2026 happi
2027 happier
2028 happiest
2029 happili
2030 harbour
2031 hard
2032 harden
2033 hardi
2034 hardli
2035 hardship
2036 harm
2037 harmless
2038 harmoni
2039 harmonis
2040 harp
2041 harri
2042 harriet
2043 harsh
2044 harshli
2045 hart_
2046 hartfield
2047 hasten
2048 hasti
2049 hastili
2050 hat
2051 hate
2052 hatr
2053 haunt
2054 hautboy
2055 have
20

2722 misinterpret
2723 misl
2724 miss
2725 misspent
2726 mistak
2727 mistaken
2728 mistress
2729 misunderstand
2730 misunderstood
2731 mitchel
2732 mix
2733 mixtur
2734 mock
2735 mode
2736 model
2737 moder
2738 modern
2739 modest
2740 modesti
2741 modestli
2742 moment
2743 momentari
2744 monarch
2745 monday
2746 money
2747 month
2748 moonlight
2749 moral
2750 moralis
2751 more
2752 moreov
2753 morn
2754 morrow
2755 mortal
2756 mortif
2757 mortifi
2758 most
2759 mostli
2760 mother
2761 motherli
2762 motion
2763 motionless
2764 motiv
2765 motto
2766 mount
2767 mourn
2768 mourner
2769 mouth
2770 move
2771 movement
2772 mr
2773 much
2774 mud
2775 muffin
2776 mule
2777 multipl
2778 multipli
2779 mum
2780 murmur
2781 muse
2782 music
2783 musician
2784 muslin
2785 must
2786 muster
2787 mute
2788 mutter
2789 mutton
2790 mutual
2791 my
2792 myself
2793 mysteri
2794 nail
2795 naivet
2796 name
2797 narr
2798 narrat
2799 narrow
2800 nash
2801 nativ
2802 natur
2803 nay
2804 near
2805 nearer
2806 ne

3472 respect
3473 respit
3474 rest
3475 restless
3476 restor
3477 restrain
3478 restraint
3479 restrict
3480 result
3481 resum
3482 retain
3483 retent
3484 reticul
3485 retir
3486 retort
3487 retract
3488 retreat
3489 retrospect
3490 return
3491 reveal
3492 reveri
3493 revers
3494 reviv
3495 revolt
3496 reward
3497 rheumat
3498 ribband
3499 ribbon
3500 rice
3501 rich
3502 richard
3503 richardson
3504 richest
3505 richli
3506 richmond
3507 rid
3508 ridden
3509 riddl
3510 ride
3511 ridicul
3512 right
3513 rightli
3514 ring
3515 ripen
3516 rise
3517 risk
3518 rival
3519 river
3520 rivet
3521 road
3522 roast
3523 rob
3524 robe
3525 robert
3526 rode
3527 romanc
3528 romant
3529 room
3530 root
3531 rose
3532 rough
3533 round
3534 roundabout
3535 roundli
3536 rous
3537 rout
3538 routin
3539 row
3540 ru
3541 rub
3542 rubber
3543 rude
3544 ruin
3545 rule
3546 rumin
3547 rumour
3548 run
3549 rung
3550 rush
3551 s
3552 sack
3553 sacr
3554 sacrif
3555 sacrific
3556 sad
3557 saddl
3558 sadli
3559 s

...

21718 bidst
21719 biserta
21720 bitumin
21721 bizanc
21722 blamest
21723 blest
21724 blis
21725 boder
21726 borea
21727 bosporu
21728 bossi
21729 briareo
21730 brigad
21731 brind
21732 bullion
21733 busiri
21734 buxom
21735 cadmu
21736 caecia
21737 calabria
21738 calamit
21739 callow
21740 calmest
21741 calumni
21742 cambalu
21743 cani
21744 caparison
21745 capitolin
21746 capricorn
21747 carnag
21748 casbeen
21749 casiu
21750 caspian
21751 castalian
21752 cathaian
21753 causey
21754 centrick
21755 cerast
21756 cerberean
21757 cere
21758 chaldaea
21759 cham
21760 champain
21761 charlemain
21762 charybdi
21763 chemick
21764 chemo
21765 chersones
21766 cherubick
21767 chimera
21768 choisest
21769 choral
21770 chrysolit
21771 cinctur
21772 circean
21773 circlet
21774 circumflu
21775 circumfus
21776 circumscrib
21777 cleombrotu
21778 climbest
21779 clo
21780 clomb
21781 cocytu
21782 coetern
21783 colick
21784 colur
21785 communic
21786 compeer
21787 compuls
21788 concav
21789 conglob
21790

22717 vallombrosa
22718 vanguard
22719 vassalag
22720 vernant
22721 vertumnu
22722 viand
22723 viceger
22724 viciou
22725 viewest
22726 vilifi
22727 virtual
22728 virtuousest
22729 volant
22730 volli
22731 volubil
22732 volumin
22733 voluptu
22734 wakest
22735 warriour
22736 washi
22737 weakest
22738 welkin
22739 wiseli
22740 woodbin
22741 worthier
22742 wrack
22743 yeanl
22744 zenith
22745 zephyru
22746 zodiack
22747 zophiel
22748 1599
22749 Br
22750 Co
22751 Ne
22752 Tu
22753 Vp
22754 Vs
22755 Y
22756 abler
22757 abou
22758 accidental
22759 actu
22760 addrest
22761 aduantag
22762 afear
22763 affabiliti
22764 affayr
22765 affraid
22766 agen
22767 alarum
22768 alass
22769 alchymi
22770 aliu
22771 anchys
22772 antonio
22773 apparrel
22774 appea
22775 apprehensiu
22776 arriu
22777 artemidoru
22778 artimedoru
22779 aswel
22780 attyr
22781 aueng
22782 auoyd
22783 aym
22784 ayr
22785 barr
22786 bastardi
22787 batchellor
22788 batchellour
22789 battel
22790 begg
22791 begger
22792 behauiour


23967 prouok
23968 prowd
23969 puft
23970 puh
23971 punisht
23972 puppet
23973 purgat
23974 pursi
23975 pursuest
23976 puzel
23977 pyrat
23978 pyrrhu
23979 quarel
23980 quickness
23981 quiddit
23982 quietu
23983 quil
23984 quillet
23985 quoth
23986 rac
23987 ranck
23988 rankli
23989 rapsidi
23990 rashness
23991 ratl
23992 raue
23993 rauell
23994 readiness
23995 reak
23996 recam
23997 recogniz
23998 recoueri
23999 redeliu
24000 reechi
24001 relatiu
24002 releef
24003 releeu
24004 relieu
24005 rellish
24006 remast
24007 remembraunc
24008 remou
24009 rendeuou
24010 renish
24011 rerul
24012 reseru
24013 respeak
24014 responsiu
24015 retook
24016 retyr
24017 reueal
24018 reuengeful
24019 reuennew
24020 reuerend
24021 reuert
24022 reuisit
24023 reuolut
24024 rew
24025 reway
24026 reynol
24027 reynoldo
24028 riual
24029 riuer
24030 riuet
24031 ro
24032 robusti
24033 romag
24034 rore
24035 rosin
24036 rosincr
24037 rosincran
24038 rossiu
24039 rouc
24040 rows
24041 sadness
24042 sai
24043 sall

25217 eschylu
25218 escul
25219 estray
25220 estuari
25221 ethnologist
25222 excelsi
25223 excelsior
25224 excrementiti
25225 excresc
25226 exhaustless
25227 exud
25228 exurg
25229 faithfulest
25230 fatherhood
25231 faucet
25232 faust
25233 feaster
25234 feb
25235 fecund
25236 feejeeman
25237 femm
25238 fernando
25239 fervor
25240 fest
25241 fester
25242 fete
25243 fetich
25244 feuillag
25245 fiat
25246 ficht
25247 filter
25248 finder
25249 firelock
25250 firemen
25251 firth
25252 flabbi
25253 flaccid
25254 flageolet
25255 flagg
25256 flagger
25257 flagston
25258 flatboatmen
25259 flauntest
25260 flex
25261 floodgat
25262 florenc
25263 fluent
25264 foetor
25265 font
25266 fop
25267 foray
25268 forcep
25269 foremen
25270 foretruck
25271 forthgoer
25272 foulest
25273 foundri
25274 fraction
25275 fractur
25276 frailest
25277 framer
25278 fratricid
25279 freest
25280 frenchwoman
25281 freshier
25282 friabl
25283 friendlili
25284 frolicsom
25285 frontest
25286 fructifi
25287 fusillad
25288 

26217 unwrap
26218 unwrit
26219 uprisen
26220 upros
26221 upsprang
26222 uranu
26223 usk
26224 ust
26225 vapor
26226 vapori
26227 vaquero
26228 vasco
26229 vaster
26230 veneer
26231 venereale
26232 venu
26233 vervain
26234 vexer
26235 victress
26236 vina
26237 virgil
26238 virginian
26239 visor
26240 vitreou
26241 viva
26242 vocalist
26243 volga
26244 voter
26245 voyagest
26246 wabash
26247 wacho
26248 wahsatch
26249 waitest
26250 wakedst
26251 walla
26252 wallabout
26253 wallachian
26254 walt
26255 warili
26256 warrante
26257 warsaw
26258 watchfir
26259 weeper
26260 welder
26261 welshman
26262 weser
26263 westernr
26264 whirr
26265 whitman
26266 whoe
26267 wholesal
26268 wielder
26269 willamett
26270 winder
26271 winterwork
26272 wisconsines
26273 wobbl
26274 wolverin
26275 womanhood
26276 woodman
26277 woodpil
26278 woolen
26279 woolli
26280 workwomen
26281 wouldist
26282 wrestler
26283 wrig
26284 xxiv
26285 xxix
26286 xxv
26287 xxvi
26288 xxvii
26289 xxviii
26290 xxx
26291 xxxi
2629

In [9]:
# 위에서 정의한 인덱스에 따라 각 텍스트를 bag of words의 형태로 바꾸어 코퍼스를 구성
corpus = [dictionary.doc2bow(text) for text in text_tokens]

print(dictionary.doc2bow(text_tokens[0]))

[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 125), (10, 46), (11, 3), (12, 12), (13, 49), (14, 34), (15, 5), (16, 13), (17, 2), (18, 60), (19, 2), (20, 11), (21, 1), (22, 4), (23, 3), (24, 2), (25, 441), (26, 3178), (27, 4), (28, 3), (29, 3), (30, 110), (31, 70), (32, 20), (33, 400), (34, 1), (35, 2), (36, 2), (37, 1), (38, 1), (39, 9), (40, 1153), (41, 108), (42, 1), (43, 126), (44, 12), (45, 185), (46, 15), (47, 4), (48, 2), (49, 50), (50, 1), (51, 56), (52, 3), (53, 3), (54, 4), (55, 89), (56, 3), (57, 3), (58, 3), (59, 2), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 3), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 2), (107,

In [15]:
# create a tf-idf model from the corpus
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)

# tf-idf 모델에 따라 문서에 사용된 단어들의 값이 산출됨
tf_idf[corpus[0]]

TfidfModel(num_docs=18, num_nnz=86391)


[(0, 0.0009857032771977616),
 (1, 0.0005195780566590131),
 (2, 0.0007950422372556407),
 (3, 0.0007950422372556407),
 (4, 0.0007950422372556407),
 (5, 0.0007950422372556407),
 (6, 0.0015900844745112815),
 (7, 0.0007950422372556407),
 (8, 0.0007950422372556407),
 (10, 0.001490311564409252),
 (11, 0.0001504512400704869),
 (12, 0.00018866785461690386),
 (14, 0.0005345589214145609),
 (15, 0.00025075206678414487),
 (17, 0.0006043811973135201),
 (18, 0.0009433392730845193),
 (19, 0.0004461182348585569),
 (20, 0.0028576793116245726),
 (21, 0.0007950422372556407),
 (22, 0.0014093640480543558),
 (23, 9.719423246147295e-05),
 (24, 0.0002709268144751991),
 (27, 0.00035805020940692735),
 (28, 0.0004063902217127986),
 (29, 0.0006691773522878354),
 (32, 0.0003144464243615064),
 (34, 0.00035234101201358896),
 (35, 0.0006043811973135201),
 (36, 0.0003813220798842416),
 (37, 0.00025978902832950656),
 (38, 0.00035234101201358896),
 (39, 0.0012191706651383959),
 (40, 0.18641700779840284),
 (42, 0.00041372

In [16]:
# a similarity measure object in tf-idf space
sims = gensim.similarities.Similarity('', tf_idf[corpus], num_features=len(dictionary))
print(type(sims))

# Each shard is stored to disk under output_prefix.shard_number.
# If you don’t specify an output prefix, a random filename in temp will be used.

<class 'gensim.similarities.docsim.Similarity'>


In [30]:
print(sims[tf_idf[corpus[4]]])

[0.00940046 0.01223217 0.00904714 0.0654783  0.99999994 0.04898324
 0.01037068 0.0084695  0.01106989 0.0309748  0.01021332 0.03754831
 0.04145316 0.22050872 0.0223679  0.02730013 0.02785867 0.16470636]


In [17]:
# create a query document and convert it to tf-idf

# query document을 위의 방법에 따라 tokenize 하기
query_doc = [w.lower() for w in tokenize("Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence.")]
print(query_doc)

['emma', 'woodhous', 'handsom', 'clever', 'and', 'rich', 'with', 'a', 'comfort', 'home', 'and', 'happi', 'disposit', 'seem', 'to', 'unit', 'some', 'of', 'the', 'best', 'bless', 'of', 'exist']


In [18]:
# query document를 위 코퍼스의 인덱스에 따라 bag of words로 변환
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)

[(226, 1), (395, 2), (615, 1), (643, 1), (884, 1), (926, 1), (1336, 1), (1493, 1), (1613, 1), (2019, 1), (2026, 1), (2129, 1), (2900, 2), (3501, 1), (3638, 1), (3825, 1), (4101, 1), (4154, 1), (4335, 1), (4601, 1), (4614, 1)]


In [19]:
# query document를 위 tf-idf 모델에 따라 표현
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

[(643, 0.030591767869065194), (884, 0.152665730756644), (1336, 0.152665730756644), (1493, 0.5706847667109823), (1613, 0.152665730756644), (2019, 0.12791083392549962), (3501, 0.014845745129688415), (4335, 0.152665730756644), (4614, 0.7507157663249546)]


In [28]:
# document similarities to query
print(sims[query_doc_tf_idf])

[4.9028519e-01 3.4097787e-03 2.3907379e-03 1.2988232e-04 1.2709189e-03
 1.3461430e-03 1.4980003e-04 4.3051751e-04 1.8733502e-03 3.0567136e-03
 7.2922016e-04 2.7870864e-03 1.2974076e-03 2.0353233e-03 1.8961601e-06
 6.0734787e-04 1.8355380e-04 4.4891909e-03]
